<img src='./images/film_logo.jpeg' style='float:left; width:200px;height:200px'/>

#  <h1><center> Capstone Project: Film Linguistics </center></h1>
#  Notebook 1 - Introduction and Data Cleaning
####  Stephen Strawbridge, Cohort #1019

## Problem Statement

I hypothesize that movie production companies are not as accurate as they could be in anticipating their reviews from viewers, due to the lack of consideration in linguistic features used in script. This project aims to create the most ideal prediction model(s), with an emphasis on script linguistics, so that production companies can anticipate their review success during the production phase and adjust budgets accordingly.

---
# Data Cleaning

---

In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Read in dataset
df = pd.read_csv('./CSVs/original_df.csv');

/Users/stephenstrawbridge/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10,12,14,16,18,20,22,24,26,28,30,32,34,44,46,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143,145,147,149,151,153,155,157,159,161,163,165,167,169,171,173,175,177,179,181,183,185,187,189,191,193,195,197,199,201,203,205,207,209,211,213,215,217,219,221,223) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Look at overall info on dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30493 entries, 0 to 30492
Columns: 224 entries, MovieID to Filler-ratio
dtypes: float64(6), int64(120), object(98)
memory usage: 52.1+ MB


In [4]:
#Isolate for columns with null values
nan_cols = [i for i in df if df[i].isnull().any()]
df[nan_cols].isnull().sum()

random_number                 58
plot_summary               28887
made_for                   27198
suspended                  30479
running_time                1877
running_time_comment       27406
country                        7
USAonly_1_other_0              7
rating_dist                  568
rating_votes                 568
rating_rank                  568
CERT_dummycode             16972
cert-west-germany          26482
genre1                       257
genre2                      9686
genre3                     19502
PrimaryGenre_dummycoded      257
dtype: int64

#### Drop unneeded rows and columns
---

In [5]:
#First, drop unecessary columns for project
df = df.drop(columns=['random_number', 'CERT_dummycode', 'cert-west-germany', 'SubActualCD', 'suspended',
                      'SubSumCD', 'plot_summary', 'made_for', 'SubDownloadsCnt.1', 'TotalWords.1', 'rating_dist'])

In [6]:
#Map the phrase 'Not provided' to object columns with null values
obj_null_cols = ['running_time', 'running_time_comment', 'country']

for col in obj_null_cols:
    df[col] = df[col].replace(np.nan, 'Not provided')

In [7]:
#Because ratings will be a primary target variable in project, rows where rating data is missing will be dropped
df = df[df['rating_votes'].notna()]
df = df[df['rating_rank'].notna()]

In [8]:
#For genres, our dataframe already has the genres dummified, so we can drop the original genre columns
df = df.drop(columns=['genre1', 'genre2', 'genre3', 'PrimaryGenre_dummycoded'])

In [9]:
#Drop the 6 null rows in the USAonly column
df = df[df['USAonly_1_other_0'].notna()]

In [10]:
#Double check that no more nulls exist in dataframe
nan_cols = [i for i in df if df[i].isnull().any()]
df[nan_cols].isnull().sum()

Series([], dtype: float64)

#### 1. Because we are specifically looking at the linguistic characteristic ratios, we will create column list of all ratio features.  The total initial number of ratios is 86.
---
After running models in the subsequent notebooks, it was noted that many of the ratios had little to no predictive value, and if anything, were a detriment to the modeling process.  For this reason, many of the ratios will be dropped.  Ratios were dropped according to either a low value count and/or low correlation with the target variable.

In [11]:
#Drop ratios deemed invaluable to models in subsequent notebooks
ratios_to_drop = ['Pronoun-ratio', 'Ppron-ratio', 'I-ratio', 'We-ratio', 'You-ratio', 'SheHe-ratio', 'They-ratio',
                 'Ipron-ratio', 'Article-ratio', 'Prep-ratio', 'Auxverb-ratio', 'Adverb-ratio', 'Conj-ratio',
                 'Compare-ratio', 'Number-ratio', 'Quant-ratio', 'Affect-ratio', 'Posemo-ratio', 'Negemo-ratio',
                 'Anx-ratio', 'CogProc-ratio', 'Insight-ratio', 'Cause-ratio', 'Discrep-ratio', 'Tentat-ratio',
                 'Certain-ratio', 'Differ-ratio', 'Percept-ratio', 'See-ratio', 'Hear-ratio', 'Feel-ratio', 'Bio-ratio',
                 'Body-ratio', 'Health-ratio', 'Ingest-ratio', 'Drives-ratio', 'Affiliation-ratio', 'Achieve-ratio',
                 'Power-ratio', 'Relativ-ratio', 'Motion-ratio', 'Space-ratio', 'Time-ratio', 'Work-ratio', 'Leisure-ratio',
                 'Home-ratio', 'Netspeak-ratio', 'Assent-ratio', 'Nonflu-ratio']

#Drop associated word count columns with these ratios, as we are only concerned with word ratios, not word counts
wordcounts_to_drop = ['Pronoun', 'Ppron', 'I', 'We', 'You', 'SheHe', 'They', 'Ipron', 'Article', 'Prep', 'Auxverb',
                      'Adverb', 'Conj', 'Compare', 'Number', 'Quant', 'Affect', 'Posemo', 'Negemo', 'Anx', 'CogProc',
                      'Insight', 'Cause', 'Discrep', 'Tentat', 'Certain', 'Differ', 'Percept', 'See', 'Hear', 'Feel',
                      'Bio', 'Body', 'Health', 'Ingest', 'Drives', 'Affiliation', 'Achieve', 'Power', 'Relativ',
                      'Motion', 'Space', 'Time', 'Work', 'Leisure', 'Home', 'Netspeak', 'Assent', 'Nonflu']

#Drop ratio's that dominated the frequency of occurence, as these ratios would become to influencing on the model
other_to_drop = ['Function', 'Function-ratio', 'Verb', 'Verb-ratio']

#Remove ratio cols, and their respective word count cols, and other ratios, from final ratios list
df = df.drop(columns=ratios_to_drop)
df = df.drop(columns=wordcounts_to_drop)
df = df.drop(columns=other_to_drop)

#### Only keep dataframe for which there is at least 1000 votes

In [12]:
df = df[df['rating_votes'] > 1000]

---

In [13]:
#Create ratio_cols list of all ratio features that we want to keep (e.g. was not dropped in previous cell)
ratio_cols = [col for col in df.columns if 'ratio' in col]

#It was noticed that a question mark was present in ratio columns
#The rows with this question mark were dropped
for col in df[ratio_cols]:
    df = df[df[col] != '?']
    
#Convert columns to floats (they are currently object types)
df[ratio_cols] = df[ratio_cols].astype(float)

In [15]:
#Check out final number of ratios used
len(ratio_cols)

35

In [16]:
#Look at final ratio list
ratio_cols

['HarmVirtue-ratio',
 'HarmVice-ratio',
 'FairnessVirtue-ratio',
 'FairnessVice-ratio',
 'IngroupVirtue-ratio',
 'IngroupVice-ratio',
 'AuthorityVirtue-ratio',
 'AuthorityVice-ratio',
 'PurityVirtue-ratio',
 'PurityVice-ratio',
 'MoralityGeneral-ratio',
 'Negative-ratio',
 'Positive-ratio',
 'Negate-ratio',
 'Adj-ratio',
 'Interrog-ratio',
 'Anger-ratio',
 'Sad-ratio',
 'Social-ratio',
 'Family-ratio',
 'Friend-ratio',
 'Female-ratio',
 'Male-ratio',
 'Sexual-ratio',
 'Reward-ratio',
 'Risk-ratio',
 'FocusPast-ratio',
 'FocusPresent-ratio',
 'FocusFuture-ratio',
 'Money-ratio',
 'Relig-ratio',
 'Death-ratio',
 'Informal-ratio',
 'Swear-ratio',
 'Filler-ratio']

#### 2. Brief feature engineering/renaming/modifications
---

In [17]:
#Create column called 'years_old' to indicate how old a certain movie is
df['years_old'] = 2021 - df['MovieYear']

#### 3. Clean country columns (note these columns were already dummified)
---

In [18]:
#Change country columns to integer types (currently stored as floats)
df['USAonly_1_other_0'] = df['USAonly_1_other_0'].astype(int)
df['USAany_1_other_0'] = df['USAany_1_other_0'].astype(int)

In [19]:
#It was noticed that a question mark was present in ratio columns
#The rows with this question mark were dropped
for col in df.columns:
    df = df[df[col] != '?']

#### 4. Drop rows for which word count is below 1,000 words
---
Because this model seeks to predict based off of word count ratios, ratios could become distorted if word counts are significantly low.  Also, integrity of the dataset is maintained as only a small percentage of movies have word counts below 1,000.


In [20]:
#Drop rows with word counts below 1,000
df = df[df['TotalWords'] > 1000]

#### 5. Drop rows for movies older than 30 years old
---
Because this model seeks to predict rating based off of English vernacular, vernacular from over 30 years ago could be significantly different than the vernacular seen and produced today.  Additionally, integrity of the dataset is maintained as the dataset still includes over 18,000 rows.

In [21]:
#Drop rows with years_old greater than 30
df = df[df['years_old'] < 30]

#### 6. Double check before saving
---

In [22]:
#Double check datatypes of columns (only select few columns should be object type)
for col in df:
    if df[col].dtype == object:
        print(col)
    else:
        continue

MovieName
running_time
running_time_comment
country


In [23]:
#Double check that no more nulls exist in dataframe
nan_cols = [i for i in df if df[i].isnull().any()]
df[nan_cols].isnull().sum()

Series([], dtype: float64)

In [24]:
#Double check shape
df.shape

(9970, 108)

#### 7. Finally, save the fully cleaned CSV to be imported in subsequent notebooks
---

In [25]:
#Save cleaned dataframe to CSV
df.to_csv('./CSVs/cleaned_df.csv')

In [26]:
#Save cleaned dataframe to Excel for reference purposes
#df.to_excel('./Excels/cleaned_df.xlsx')

In [27]:
#Save cleaned dataframe to Excel for Tableau purposes
#df.to_excel('./Excels/tableau_df.xlsx')